Car Pipeline:
Car Detection (YOLO) -> detect and classify driver and seatbelt (Custom) Not Done Yet -> license plate detection(YOLO) -> OCR (pytesseract)


In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip -q install ultralytics
    !sudo apt install tesseract-ocr libtesseract-dev
    !pip -q install pytesseract
    


In [ ]:
from ultralytics import YOLO
import pytesseract
from pathlib import Path


license_plate_detection_model = YOLO('yolov11x-license-plate.pt')

car_detection_model = YOLO("yolov11n.pt")
    


In [ ]:
def detect_cars_and_plates(image_path: Path | str):
    # First, detect cars in the image
    car_results = car_detection_model.predict(source=image_path)
    output_plates = []
    for car in car_results[0].boxes:
        # Extract car bounding box
        x1, y1, x2, y2 = map(int, car.xyxy[0])
        car_image = car_results[0].orig_img[y1:y2, x1:x2]
        
        plate_results = license_plate_detection_model.predict(source=car_image)
        
        for plate in plate_results[0].boxes:
            # Extract license plate bounding box
            px1, py1, px2, py2 = map(int, plate.xyxy[0])
            license_plate_image = car_image[py1:py2, px1:px2]
            # ocr 
            ocr_result = pytesseract.image_to_string(license_plate_image)
            print("Detected License Plate Text:", ocr_result)
            output_plates.append(ocr_result)
    return output_plates

In [ ]:
images = ['./bmwalleng.png']
for img_path in images:
    print(f"Processing image: {img_path}")
    plates = detect_cars_and_plates(img_path)
    print(f"Detected plates: {plates}")